In [124]:
import numpy as np
import pandas as pd
import requests as rq
import math
from scipy import stats
import xlsxwriter

In [125]:
from secret_case import API_TOKEN

In [126]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [127]:
url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":"AAPL"}

headers = {
    'x-api-key': API_TOKEN
    }

response = rq.get(url, headers=headers, params=querystring).json()

print(response)

{'quoteResponse': {'result': [{'language': 'en-US', 'region': 'US', 'quoteType': 'EQUITY', 'typeDisp': 'Equity', 'quoteSourceName': 'Nasdaq Real Time Price', 'triggerable': True, 'customPriceAlertConfidence': 'HIGH', 'currency': 'USD', 'marketState': 'PREPRE', 'regularMarketChangePercent': -0.88974994, 'regularMarketPrice': 170.43, 'epsTrailingTwelveMonths': 5.97, 'epsForward': 6.22, 'epsCurrentYear': 5.73, 'priceEpsCurrentYear': 29.743454, 'sharesOutstanding': 15634199552, 'bookValue': 3.852, 'fiftyDayAverage': 182.1924, 'fiftyDayAverageChange': -11.762405, 'fiftyDayAverageChangePercent': -0.064560354, 'twoHundredDayAverage': 165.9458, 'twoHundredDayAverageChange': 4.484192, 'twoHundredDayAverageChangePercent': 0.027022026, 'marketCap': 2664536473600, 'forwardPE': 27.400322, 'priceToBook': 44.244545, 'sourceInterval': 15, 'exchangeDataDelayedBy': 0, 'averageAnalystRating': '2.0 - Buy', 'tradeable': False, 'cryptoTradeable': False, 'postMarketChangePercent': -0.0234661, 'postMarketTime

In [128]:
def chunks (lst, n):
    """Yield succisve n-sized chunks form lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [129]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
dataframes = []
rows= []

In [130]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(f"Processing batch {i} of symbols")

Processing batch 0 of symbols
Processing batch 1 of symbols
Processing batch 2 of symbols
Processing batch 3 of symbols
Processing batch 4 of symbols
Processing batch 5 of symbols


In [152]:
dataframes = []

for symbol_string in symbol_strings:
    batch_api_url = "https://yfapi.net/v6/finance/quote"
    
    querystring = {"symbols": symbol_string}
    
    headers = {
        'x-api-key': API_TOKEN
    }

    data = rq.get(batch_api_url, headers=headers, params=querystring).json()

    rows = []
    for result in data.get('quoteResponse', {}).get('result', []):
        symbol = result.get('symbol', np.nan)
        regular_market_price = float(result.get('regularMarketPrice', np.nan))
        market_cap = float(result.get('marketCap', np.nan))

        rows.append([symbol, regular_market_price, market_cap, np.nan])

    dataframe = pd.DataFrame(rows, columns=my_columns)
    dataframes.append(dataframe)

final_dataframe = pd.concat(dataframes, ignore_index=True)

In [153]:
numeric_columns = final_dataframe.columns.difference(['Ticker', 'Number of Shares to Buy'])
final_dataframe[numeric_columns] = final_dataframe[numeric_columns].fillna(-1).astype(float)
final_dataframe.head()

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,110.50,3.233086e+10,NaN
1,AAL,12.61,8.238895e+09,NaN
2,AAP,54.82,3.260162e+09,NaN
3,AAPL,170.43,2.664536e+12,NaN
4,ABBV,153.13,2.702821e+11,NaN


In [154]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That is not a number! \nPlease try again:")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

In [155]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] =  math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,110.50,3.233086e+10,179559.0
1,AAL,12.61,8.238895e+09,1573455.0
2,AAP,54.82,3.260162e+09,361934.0
3,AAPL,170.43,2.664536e+12,116418.0
4,ABBV,153.13,2.702821e+11,129571.0
...,...,...,...,...
499,YUM,121.98,3.418014e+10,162660.0
500,ZBH,110.76,2.314485e+10,179137.0
501,ZBRA,225.91,1.159786e+10,87828.0
502,ZION,33.17,4.913969e+09,598169.0


In [157]:
writer = pd.ExcelWriter('reccomended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Reccomended Trades', index = False)

In [158]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [1]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Reccomended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])

NameError: name 'string_format' is not defined

In [161]:
writer.close()